In [4]:
import numpy as np
import pandas as pd
from pprint import pprint
from movie import *
import pickle
from multiprocessing import Pool
from datetime import datetime
from pprint import pprint

from sklearn.feature_selection import VarianceThreshold
import sklearn.datasets as datasets
from sklearn.feature_selection import SelectPercentile,f_classif, f_regression,SelectKBest
from sklearn.feature_selection import chi2
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV

In [36]:
with open('name_url.pkl','rb') as f:
    name_info=pickle.load(f)

print(name_info['Bubble Boy(2001)'])

with open('movie_info.pkl','rb') as f:
    movie_info=pickle.load(f)

print(movie_info['Bubble Boy(2001)']['summary'])

with open('','rb') as f:
    

{'Release Date': datetime.datetime(2001, 8, 24, 0, 0), 'url': 'https://www.the-numbers.com/movie/Bubble-Boy', 'Production Budget': 13000000, 'Domestic Gross': 5002310, 'Worldwide Gross': 5002310}
{'Budget': 13000000, 'Domestic Releases': {'Wide': datetime.datetime(2001, 8, 24, 0, 0)}, 'MPAA Rating': 'PG-13 for language and crude sexual humor', 'Source': 'Original Screenplay', 'Genre': 'Comedy', 'Production Method': ['Live Action'], 'Creative Type': 'Contemporary Fiction', 'ranking on other records': []}


## Data preprocessing


In [2]:
def get_roles():
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    role_set=set()
    for name,info in movie_info.items():
        if 'production' in info['cast'].keys():
            for person in info['cast']['production']:
                role_set.add(person['role'])
    
    return list(role_set)

In [34]:
print(len(get_roles()))

1965


In [3]:
with open('name_url.pkl','rb') as f:
    name_dic=pickle.load(f)

print(name_dic['Deep Throat(1972)'])

{'Release Date': datetime.datetime(1972, 6, 30, 0, 0), 'url': 'https://www.the-numbers.com/movie/Deep-Throat', 'Production Budget': 25000, 'Domestic Gross': 45000000, 'Worldwide Gross': 45000000}


In [4]:
import math
with open('movie_info.pkl','rb') as f:
    movie_info=pickle.load(f)
    
with open('name_url.pkl','rb') as f:
    movie_budget=pickle.load(f)
    
def get_person_info_helper(name, movie_info=movie_info, movie_budget=movie_budget):
    
    print(name)
    person_name_info=dict()
    #cast_url=value+"#tab=cast-and-crew"
    cast_dic=movie_info[name]['cast']
    cast_list=['leading_members', 'supporting_cast', 'cameos', 'uncategorized', 'production']
    for cate in cast_list:
        if cate in cast_dic.keys():
            for person in cast_dic[cate]:
                person_name=person['name']
                if person_name not in person_name_info.keys():
                    acting_response = requests.get(person['url']+"#tab=acting")
                    technical_response = requests.get(person['url']+"#tab=technical")
                    count=0
                    total_roi=0.0
                    
                    if acting_response.status_code != 404:
                        acting_dic = parse_person_page(acting_response.content, "acting")
                        for item in acting_dic['All Acting Credits']:
                            title=item['Title']+'('+str(item['Release Date'].year)+')'
                            if item['Release Date']<datetime.now() and title in movie_budget.keys():
                                count+=1
                                budget=movie_budget[title]['Production Budget']
                                total_roi+=(item['Worldwide Box Office']-budget)/budget
        
                    if technical_response.status_code != 404:
                        technical_dic = parse_person_page(technical_response.content, "technical")
                        for item in technical_dic['All Technical Credits']:
                            title=item['Title']+'('+str(item['Release Date'].year)+')'
                            if item['Release Date']<datetime.now() and title in movie_budget.keys():
                                count+=1
                                budget=movie_budget[title]['Production Budget']
                                total_roi+=(item['Worldwide Box Office']-budget)/budget
                    
                    person_name_info[person_name]=math.log(total_roi/(count+0.1)+1)
    return person_name_info

In [9]:
get_person_info_helper('Black Panther(2018)')

Black Panther(2018)


{'Andy Gill': 1.0013089575265794,
 'Andy Serkis': 1.0516114223571904,
 'Angela Bassett': 0.9074115230288603,
 'Atandwa Kani': 1.817018306553155,
 'Chadwick Boseman': 0.9258571075190921,
 'Claudia Castello': 1.6969120416606482,
 'Connie Chuene': 1.817018306553155,
 'Danai Gurira': 1.0270141903092307,
 'Daniel Kaluuya': 2.56854654567628,
 'Dave Jordan': 1.1073082703465975,
 'David J. Grant': 1.5632819256684796,
 'Florence Kasumba': 1.7607751439389416,
 'Forest Whitaker': 1.5543610189334476,
 'Geoffrey Baumann': 1.6585379021664026,
 'Hannah Beachler': 2.0216914021567236,
 'Isaach de Bankole': 1.1905897668775083,
 'Jack Kirby': 1.3030323681410725,
 'Jeffrey Chernov': 0.983918368199135,
 'Joe Robert Cole': 1.817018306553155,
 'John Kani': 1.6901001710034822,
 'Kevin Feige': 1.266988323725113,
 'Letitia Wright': 1.600351011798708,
 'Louis D’Esposito': 1.418997652500118,
 'Ludwig Goransson': 1.754136761805202,
 "Lupita Nyong'o": 1.8521047930581473,
 'Martin Freeman': 1.114981861765203,
 'Mich

In [7]:
def get_all_person_urls():
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    total_dic=dict()
    keys=list(movie_info.keys())
    pool = Pool(processes=10)
    pool_outputs = pool.map(get_person_info_helper,keys)
    pool.close()
    pool.join()
    
    for item in pool_outputs:
        for key, value in item.items():
            total_dic[key]=value
    
    with open('person_info.pkl','wb') as f:
        pickle.dump(total_dic, f)
        
    return

In [8]:
get_all_person_urls()

Transformers(2007)
Avatar(2009)
The Island(2005)
Kubo and the Two Strings(2016)
The Finest Hours(2016)
The Departed(2006)
Mars Attacks!(1996)
Kangaroo Jack(2003)
Assassins(1995)
Rebound(2005)
Zoolander 2(2016)
The Ghost Writer(2010)
The Happening(2008)
Ronin(1998)
The Wild(2006)
Charlie's Angels(2000)
All the Pretty Horses(2000)
Wonder Woman(2017)
Defiance(2009)
Bangkok Dangerous(2008)
Nixon(1995)
Pirates of the Caribbean: On Stranger Tides(2011)
Escape Plan(2013)
Hannibal Rising(2007)
Burlesque(2010)
The Story of Us(1999)
Mulan(1998)
The Host(2013)
Dredd(2012)
London Has Fallen(2016)
Tropic Thunder(2008)
Walking with Dinosaurs(2013)
Killer Elite(2011)
How Do You Know?(2010)
Knight and Day(2010)
Basic(2003)
Hard Rain(1998)
The Ghost and the Darkness(1996)
Hart's War(2002)
The Rum Diary(2011)
Changeling(2008)
Avengers: Age of Ultron(2015)
The Girl with the Dragon Tattoo(2011)
Rollerball(2002)
Thirteen Days(2000)
The Legend of Hercules(2014)
Star Wars Ep. I: The Phantom Menace(1999)
Froz

Tooth Fairy(2010)
Invictus(2009)
Ice Age(2002)
The Mothman Prophecies(2002)
Licence to Kill(1989)
Anastasia(1997)
Rat Race(2001)
Apollo 13(1995)
Real Steel(2011)
One for the Money(2012)
The Devil's Own(1997)
Moulin Rouge(2001)
Pirates of the Caribbean: Dead Man’s Chest(2006)
Troy(2004)
Madagascar(2005)
State of Play(2009)
Domestic Disturbance(2001)
The A-Team(2010)
15 Minutes(2001)
The Matrix(1999)
Grown Ups(2010)
Man of Steel(2013)
The Last Airbender(2010)
Lord of War(2005)
K-19: The Widowmaker(2002)
Ready Player One(2018)
Les Miserables(2012)
The Smurfs 2(2013)
Shark Tale(2004)
Flags of Our Fathers(2006)
3:10 to Yuma(2007)
X-Men(2000)
My Favorite Martian(1999)
Conan the Barbarian(2011)
The Divergent Series: Allegiant(2016)
The Chronicles of Narnia: Prince Caspian(2008)
K-PAX(2001)
Super Mario Bros.(1993)
The Three Musketeers(2011)
Kate and Leopold(2001)
Hero(1992)
Bee Movie(2007)
The Santa Clause 2(2002)
Man on a Ledge(2012)
The Sentinel(2006)
Wanted(2008)
The Amazing Spider-Man(2012

Watchmen(2009)
2012(2009)
Erin Brockovich(2000)
The Rundown(2003)
Peter Rabbit(2018)
Planet of the Apes(2001)
Arthur Christmas(2011)
Zathura(2005)
The Simpsons Movie(2007)
The Accountant(2016)
Stealth(2005)
Terminator Salvation(2009)
The Bucket List(2007)
xXx: Return of Xander Cage(2017)
The Missing(2003)
National Treasure(2004)
Fool's Gold(2008)
Funny People(2009)
The Client(1994)
Green Lantern(2011)
The Muppets(2011)
How to Lose a Guy in 10 Days(2003)
Django Unchained(2012)
Goosebumps(2015)
Interview with the Vampire: The Vampire Chronicles(1994)
Meet Joe Black(1998)
The Incredible Hulk(2008)
The Kingdom(2007)
Dumb and Dumber To(2014)
Elektra(2005)
Prince of Persia: Sands of Time(2010)
Hulk(2003)
Cats & Dogs: The Revenge of Kitty Galore(2010)
Little Fockers(2010)
Yes Man(2008)
Patriot Games(1992)
Astro Boy(2009)
Rambo III(1988)
Sabrina(1995)
True Lies(1994)
Final Fantasy: The Spirits Within(2001)
Non-Stop(2014)
I, Frankenstein(2014)
The Twilight Saga: Breaking Dawn, Part 2(2012)
Tran

Something's Gotta Give(2003)
Babylon A.D.(2008)
R.I.P.D.(2013)
Star Trek: Insurrection(1998)
Couples Retreat(2009)
Crimson Tide(1995)
The Count of Monte Cristo(2002)
Big Fish(2003)
Four Christmases(2008)
Frankenstein(1994)
The Prestige(2006)
The Mummy Returns(2001)
Goldeneye(1995)
Wolf(1994)
Face/Off(1997)
Gangs of New York(2002)
Resident Evil: Apocalypse(2004)
Jupiter Ascending(2015)
The Twilight Saga: Breaking Dawn, Part 1(2011)
Underworld: Awakening(2012)
The Book of Life(2014)
The Italian Job(2003)
Flightplan(2005)
The Lake House(2006)
The Glimmer Man(1996)
The Lego Ninjago Movie(2017)
Multiplicity(1996)
Alien: Covenant(2017)
The Pledge(2001)
Absolute Power(1997)
Dinosaur(2000)
Disclosure(1994)
Dracula Untold(2014)
Bedtime Stories(2008)
City of Angels(1998)
X-Men: Apocalypse(2016)
Predators(2010)
Cinderella(2015)
The Odd Life of Timothy Green(2012)
Logan(2017)
Daddy Day Care(2003)
Firewall(2006)
The General's Daughter(1999)
Road to Perdition(2002)
The Producers: The Movie Musical(2

The Brothers Grimm(2005)
Miracle(2004)
Rush Hour 2(2001)
Last Vegas(2013)
Terminator 3: Rise of the Machines(2003)
There Be Dragons(2011)
Van Helsing(2004)
Queen of the Desert(2017)
Sherlock Holmes(2009)
Exit Wounds(2001)
Life, or Something Like It(2002)
Mike and Dave Need Wedding Dates(2016)
A Dog’s Purpose(2017)
Head of State(2003)
Step Up 3D(2010)
Step Up Revolution(2012)
Ice Age: Dawn of the Dinosaurs(2009)
Drag Me To Hell(2009)
It(2017)
How the Grinch Stole Christmas(2000)
Tomorrowland(2015)
The Cabin in the Woods(2012)
The Wedding Planner(2001)
Ironclad(2011)
What Lies Beneath(2000)
Dawn of the Dead(2004)
Red Sky(2015)
Waterloo(1970)
The Adventurer: The Curse of the Midas Box(2014)
Snakes on a Plane(2006)
For Your Eyes Only(1981)
Mission: Impossible 2(2000)
Rio(2011)
Secondhand Lions(2003)
The Royal Tenenbaums(2001)
Reagan(2016)
Ishtar(1987)
Dangerous Liaisons(2012)
The Lucky One(2012)
Memoirs of an Invisible Man(1992)
Star Trek IV: The Voyage Home(1986)
Amelia(2009)
Never Back D

...

The Omen(1976)
Ca$h(2010)
Lady in White(1988)
Hustle & Flow(2005)
Driving Lessons(2006)
See No Evil(2006)
The Wash(2001)
Diamonds(1999)
Without Men(2011)
The Good Girl(2002)
Treading Water(2015)
Dogma(1999)
Bella(2007)
Cidade de Deus(2003)
Trudno byt bogom(2015)
The Big Sick(2017)
The Call(2013)
3 Strikes(2000)
Human Traffic(2000)
The Visitor(2008)
Madadayo(1998)
Bon Cop, Bad Cop(2006)
Carol(2015)
The Boondock Saints 2: All Saints Day(2009)
The Dead Girl(2006)
Friday the 13th Part VII: The New Blood(1988)
The Cooler(2003)
Artie Lange's Beer League(2006)
The Misfits(1961)
Pat Garrett and Billy the Kid(1973)
Prom(2011)
Son of Rambow: A Home Movie(2008)
The Night Listener(2006)
The Wizard of Oz(1939)
Camping Sauvage(2004)
Barfi(2012)
The Jerky Boys(1995)
The Book of Mormon Movie, Volume 1: The Journey(2003)
Hamlet(2000)
The Opposite Sex(2015)
El orfanato(2007)
Dear Frankie(2005)
Bug(2007)
A Christmas Story(1983)
Operation Chromite(2016)
Class of 1984(1982)
Le nom des gens(2011)
The Rules 

Pendulum(2001)
Trust(2011)
A Funny Thing Happened on the Way to the Forum(2014)
The Players Club(1998)
Teen Wolf Too(1987)
The Skeleton Twins(2014)
The Other End of the Line(2008)
Slacker Uprising(2007)
8: The Mormon Proposition(2010)
Sleep Dealer(2009)
The Collector(2009)
The Fourth Kind(2009)
Eye of the Dolphin(2007)
Phantasm II(1988)
The Innkeepers(2012)
The Outsiders(1983)
Hatchet(2007)
The Walking Deceased(2015)
Phat Girlz(2006)
Underdogs(2013)
Woman Thou Art Loosed(2004)
Billy Elliot(2000)
My Name is Bruce(2008)
Real Women Have Curves(2002)
The Way Way Back(2013)
An Everlasting Piece(2000)
The Disappearance of Alice Creed(2010)
Gallipoli(1981)
Daisy Winters(2017)
Water(2006)
Modern Times(1936)
The Salon(2007)
True Bromance(2011)
Don’t Think Twice(2016)
The Curse of Downers Grove(2015)
One Flew Over the Cuckoo's Nest(1975)
20 Feet From Stardom(2013)
Il conformista(2014)
Anatomie(2000)
Stolen Summer(2002)
The Calling(2009)
Alien Uprising(2013)
Road Hard(2015)
Silent Movie(1976)
Foo

Niagara(1953)
Creature(2011)
Jane Austen's Mafia(1998)
How Green Was My Valley(1941)
The Pirate(1948)
Addicted(2014)
The History Boys(2006)
Freeway(1996)
Da Sweet Blood of Jesus(2015)
Luminarias(2000)
Never Again(2002)
POM Wonderful Presents: The Greatest Movie Ever Sold(2011)
Paranormal Activity: The Ghost Dimension(2015)
To Kill A Mockingbird(1962)
Love's Abiding Joy(2006)
Dead Alive(1993)
Silent House(2012)
Shichinin no samurai(1956)
Chain Letter(2010)
Unknown(2006)
I Origins(2014)
Civil Brand(2003)
Travellers and Magicians(2005)
Ed and his Dead Mother(1993)
Monsters(2010)
Saw(2004)
Lonesome Jim(2006)
Midnight Cowboy(1969)
Deadline(2009)
Sublime(2007)
Dysfunctional Friends(2012)
Billy: The Early Years of Billy Graham(2008)
Tim and Eric's Billion Dollar Movie(2012)
Airplane!(1980)
Deceptive Practice: The Mysteries and Mentors of Ricky Jay(2013)
Sugar Hill(1994)
Antarctica: A Year on Ice(2014)
Life of Brian(1979)
Independence Daysaster(2013)
The Battle of Shaker Heights(2003)
Against 

Forever My Girl(2018)
She's Gotta Have It(1986)
Some Guy Who Kills People(2012)
Midnight Cabaret(2013)
Steppin: The Movie(2008)
All You Need Is Love(2014)
Zombies of Mass Destruction(2010)
Snow White: A Deadly Summer(2012)
Kids(1995)
Another Earth(2011)
Truth or Die(2012)
Sex with Strangers(2002)
Beyond the Black Rainbow(2012)
Sunshine Cleaning(2009)
Cinderella(1950)
Teeth and Blood(2015)
Black Rock(2013)
The Squid and the Whale(2005)
Central do Brasil(1998)
Waltz with Bashir(2008)
Strangers with Candy(2006)
Pocketful of Miracles(1961)
Walking to Paris(2015)
Anderson's Cross(2010)
Dracula: Pages from a Virgin's Diary(2003)
Fifty Shades of Black(2016)
Amidst the Devil's Wings(2015)
Death Calls(2010)
Sparrows Dance(2013)
Creepshow 2(1987)
The Sleepwalker(2014)
Boynton Beach Club(2006)
Freakonomics(2010)
Kissing Jessica Stein(2002)
Compliance(2012)
Heroes of Dirt(2015)
Hidden Away(2014)
Shortbus(2006)
My Last Day Without You(2013)
In Cold Blood(1967)
The Nun's Story(1959)
Graduation Day(1

El Mariachi(1993)
Primer(2004)
Cavite(2006)
The Mongol King(2004)
Following(1999)
Return to the Land of Wonders(2005)
Signed Sealed Delivered(2015)
A Plague So Pleasant(2015)
My Date With Drew(2005)
Rabbit Hole(2010)


In [61]:
import math
with open('movie_info.pkl','rb') as f:
    movie_info=pickle.load(f)
    
with open('name_url.pkl','rb') as f:
    movie_budget=pickle.load(f)
    
def get_company_info_helper(name, movie_info=movie_info, movie_budget=movie_budget):
    
    company_name_info=dict()
    cast_dic=movie_info[name]['summary']
    if name in movie_budget.keys() and movie_budget[name]['Release Date']<datetime.now():
        if 'Production Companies' in cast_dic.keys():
            for company in cast_dic['Production Companies']:
                #print(name, company.strip())
                budget=movie_budget[name]['Production Budget']
                roi=(movie_budget[name]['Worldwide Gross']-budget)/budget
                if roi<=5:
                    company_name_info[company.strip()]=company_name_info.get(company.strip(),[])+[roi]
    
    return company_name_info

In [58]:
def get_all_company_info():
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    total_dic=dict()
    keys=list(movie_info.keys())
    pool = Pool(processes=10)
    pool_outputs = pool.map(get_company_info_helper,keys)
    pool.close()
    pool.join()
    
    for item in pool_outputs:
        for key, value in item.items():
            total_dic[key.strip()]=total_dic.get(key.strip(),[])+value
    
    #pprint(total_dic)
    company_info=dict()
    for key,value in total_dic.items():
        company_info[key]=math.log(np.sum(np.array(value))/(len(value)+0.1) + 1)
    
    with open('company_info.pkl','wb') as f:
        pickle.dump(company_info, f)
        
    return    

In [62]:
get_all_company_info()

In [63]:
import operator
with open('company_info.pkl', 'rb') as f:
    company_info=pickle.load(f)
    
sorted_company=sorted(company_info.items(), key=lambda d: d[1])
print(sorted_company[-20:])

[('Hungarian National Film Fund', 1.6632104998171624), ('Claims Conference', 1.6632104998171624), ('Cristal Pictures', 1.6727216238999323), ('East Light Film', 1.6727216238999323), ('CG Films', 1.6727216238999323), ('Many Rivers', 1.6731232856535507), ('Boy of the Year', 1.6731232856535507), ('Atlas Productions', 1.6833841908174774), ('Classico Productions', 1.6833841908174774), ("Gettin' Rad Productions", 1.6847106605183402), ('Opus Film', 1.6937761685354085), ('Portobello Pictures', 1.6937761685354085), ('Gil Friesen', 1.6951928337471163), ('Temolo Productions', 1.6951928337471163), ('Windy Hill Pictures', 1.6965909349470847), ('Salamander Pictures', 1.6965909349470847), ('Pam Williams Productions', 1.6965909349470847), ('Michael Bay', 1.6966805509690057), ('Sahdy Acres', 1.7098744220170143), ('Pitt Bull', 1.7098744220170143)]


In [ ]:
temp_list=get_roles()
def convert_helper(name, product_list=temp_list):
    print(name)
    total_dic=dict()
    with open('movie_info.pkl', 'rb') as f:
        movie_info=pickle.load(f)
    
    info=movie_info[name]
    total_dic['name']=name
    sum_list=['Franchise', 'Genre', 'Creative Type', 'Budget', 'Keywords', 'Production Companies','Production Countries', 'Production Method', 'Video Release']
    for cate in sum_list:
        cate_name='_'.join(cate.lower().split())
        if cate in info['summary'].keys():
            total_dic[cate_name]=info['summary'][cate]
        else:
            total_dic[cate_name]='-'
    
    for item in info['box_office']['demostic']:
        if 'Demostic' in item['Record']:
            total_dic['demostic']=item['Revenue']
            break
    
    international_list=['International', 'Worldwide']
    for cat in international_list:
        for item in info['international'][cat+' Box Office']:
            if cat in item['Record']:
                total_dic[cat.lower()]=item['Revenue']
                break
                
        if cat.lower() not in total_dic.keys():
            total_dic[cat.lower()]='-'
            
    cast_list=['cameos', 'supporting_cast', 'leading_members', 'supporting_cast']
    for cate in cast_list:
        if cate in info['cast'].keys():
            total_dic[cate]=[a['name'] for a in info['cast'][cate]]
        else:
            total_dic[cate]=[]
    
    if 'production' in info['cast'].keys():
        item=info['cast']['production']
        for role in product_list:
            total_dic[role]=[a['name'] for a in item if a['role']==role]
            
    return total_dic

In [ ]:
#convert the dictionary to pandas
total_dic=dict()
with open('name_url.pkl','rb') as f:
    name_dic=pickle.load(f)
    
keys= list(name_dic.keys())
pool = Pool(processes=10) 
pool_outputs = pool.map(convert_helper,keys)
pool.close()
pool.join()

for item in pool_outputs:
    for key, value in item.items():
        total_dic[key]=total_dic.get(key,[])+[value]
    
pdf=pd.DataFrame(data=total_dic, columns=total_dic.keys())
print(pdf) 

## Feature Selection


In [ ]:
#The importance of the feaures


In [21]:
#Select the important features
from tqdm import tqdm
for i in tqdm(range(10)):
    pass

100%|██████████| 10/10 [00:00<00:00, 6373.35it/s]


## Visualize the network


In [5]:
import networkx as nx

from bokeh.io import show, output_notebook
from bokeh.plotting import figure
from bokeh.models.graphs import from_networkx

G=nx.karate_club_graph()

plot = figure(title="Networkx Integration Demonstration", x_range=(-1.1,1.1), y_range=(-1.1,1.1))

graph = from_networkx(G, nx.spring_layout, scale=2, center=(0,0))
plot.renderers.append(graph)

output_notebook()
show(plot)

Loading BokehJS ...